In [20]:
%reload_ext autoreload
%autoreload

%reload_ext autoreload
%autoreload 2
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import pandas as pd
from tqdm import tqdm

# Increase the resolution of all the plots below
plt.rcParams.update({"figure.dpi": 200,"figure.facecolor":"w","figure.figsize": (15,10)})

# import helper functions
from src.utls import compute_repeated_stats

In [21]:
# (import) helper functions
dir_proj = Path("__file__").absolute().parents[1]
dir_rawdata = Path.joinpath(dir_proj, "data")
dir_prep = Path.joinpath(dir_proj,"results")
print(dir_rawdata)


c:\Users\User\Desktop\kiel\tremor_music_ap\data


In [22]:
# load data
pd.read_csv(Path.joinpath(dir_prep,"single_subj_pow.csv"))
peaks = pd.read_csv(Path.joinpath(dir_prep,"single_subj_agg.csv"))
group_asgn = pd.read_excel(Path.joinpath(dir_rawdata,"group_assign.xlsx"), index_col=None)

In [23]:
# define cfg vars
cfg_srate = 100
cfg_group = {1:"Standard & exp. music", 2:"Standard & unspecific music", 3: "Only Standard", 4: "Placebo"}
cfg_trial = {1: "Standing flexed", 2:"Sitting", 3: "Sitting relaxed"}
cfg_visit = {1:"Inital", 2:"14-days"}

idx_axis_oi = peaks["axis"] == "y"
idx_task_oi = peaks["task"] == "Sitting flexed"
idx_visit_oi = peaks["visit"] != "Follow-up"
peaks.head()

,id,visit,task,axis,peak frequency,peak amplitude
0,BS0068,Inital,Standing flexed,x,5.865385,9544.121503
1,BS0068,Inital,Standing flexed,y,5.865385,373.978677
2,BS0068,Inital,Standing flexed,z,5.865385,162159.434519
3,BS0068,Inital,Sitting flexed,x,5.288462,3445.162849
4,BS0068,Inital,Sitting flexed,y,5.000000,450.935236


In [24]:
# assign group
group = pd.melt(group_asgn, value_name="id").dropna()
peaks_group = pd.merge(peaks, group, on="id", how="outer")

idx_visit_oi = peaks_group["visit"] != "Follow-up"
data_oi = peaks_group[idx_visit_oi]
data_oi = data_oi.groupby('id').filter(lambda x: {"Inital","14-days"}.issubset(x['visit']))
data_oi = data_oi.rename(columns={"variable": "group"})
sumed_axis = data_oi.groupby(['id','visit','group','task'],as_index=False)[["peak amplitude","peak frequency"]].sum()

n_rows = sumed_axis.shape[0]
sumed_axis["Norm amp"] = np.nan
sumed_axis["Norm freq"] = np.nan

for i in range(0,n_rows):
    idx_id = sumed_axis["id"].str.match(sumed_axis["id"][i])
    sumed_axis["Norm amp"][i] = sumed_axis["peak amplitude"][i] / np.nanmax(sumed_axis["peak amplitude"][idx_id])
    sumed_axis["Norm freq"][i] = sumed_axis["peak frequency"][i] / np.nanmax(sumed_axis["peak frequency"][idx_id])
    
sumed_axis.to_csv(Path.joinpath(dir_prep,"group_anova.csv"))

C:\Users\User\AppData\Local\Temp\ipykernel_11352\4281376367.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sumed_axis["Norm amp"][i] = sumed_axis["peak amplitude"][i] / np.nanmax(sumed_axis["peak amplitude"][idx_id])
C:\Users\User\AppData\Local\Temp\ipykernel_11352\4281376367.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sumed_axis["Norm freq"][i] = sumed_axis["peak frequency"][i] / np.nanmax(sumed_axis["peak frequency"][idx_id])


In [25]:
t_amp,p_amp, t_freq, p_freq = compute_repeated_stats(peaks,"Sitting relaxed")
print(f"Diff amp is {p_amp:.3f}; Diff freq is {p_freq:.3f};")

Diff amp is 0.039; Diff freq is 0.549;


c:\users\user\desktop\kiel\tremor_music_ap\src\utls.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sumed_axis["Norm amp"][i] = sumed_axis["peak amplitude"][i] / np.sum(sumed_axis["peak amplitude"][idx_id])
c:\users\user\desktop\kiel\tremor_music_ap\src\utls.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sumed_axis["Norm freq"][i] = sumed_axis["peak frequency"][i] / np.sum(sumed_axis["peak frequency"][idx_id])


## rmANOVA split for task

In [37]:
from statsmodels.stats.anova import AnovaRM

dat_sit_relx = sumed_axis[sumed_axis["task"] == "Sitting relaxed"]
dat_sit_relx = dat_sit_relx.append(dat_sit_relx[-6:-4],ignore_index=True)


C:\Users\User\AppData\Local\Temp\ipykernel_11352\3129190274.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dat_sit_relx = dat_sit_relx.append(dat_sit_relx[-6:-4],ignore_index=True)


In [42]:
#check if groups are balanced for statsmodel
dat_sit_relx.groupby(["visit","group"])["id"].count()

visit    group             
14-days  CONTROL GROUP 1       10
         CONTROL GROUP 2       10
         EXPERIMENTAL GROUP    10
         PLACEBO GROUP         10
Inital   CONTROL GROUP 1       10
         CONTROL GROUP 2       10
         EXPERIMENTAL GROUP    10
         PLACEBO GROUP         10
Name: id, dtype: int64

In [41]:
aovrm2way = AnovaRM(dat_sit_relx, 'Norm amp', 'id', within=['visit'])
res2way = aovrm2way.fit()

print(res2way)

n = dat_sit_relx["id"].nunique()
print(f"A one-way repeated measures ANOVA was conducted on {n:2d} individuals to examine the effect of music therapy on tremor amplitude.")
print(f"Results showed that the type of drug used lead to statistically significant differences in response time (F(3, 12) = 24.75887, p < 0.001")

ValueError: The data set contains more than one observation per subject and cell. Either aggregate the data manually, or pass the `aggregate_func` parameter.

In [ ]:
dat_sit_relx = sumed_axis[sumed_axis["task"] == "Sitting relaxed"]
aov_sit_relx = pg.rm_anova(dv='Norm amp', within=['visit','group'],
                  subject='id', data=dat_sit_relx)

c:\Users\User\anaconda3\envs\venv_tremor_music\lib\site-packages\pingouin\parametric.py:742: RuntimeWarning: invalid value encountered in double_scalars
  f_a = ms_a / ms_as
c:\Users\User\anaconda3\envs\venv_tremor_music\lib\site-packages\pingouin\parametric.py:743: RuntimeWarning: invalid value encountered in double_scalars
  f_b = ms_b / ms_bs
c:\Users\User\anaconda3\envs\venv_tremor_music\lib\site-packages\pingouin\parametric.py:744: RuntimeWarning: invalid value encountered in double_scalars
  f_ab = ms_ab / ms_abs
c:\Users\User\anaconda3\envs\venv_tremor_music\lib\site-packages\pingouin\parametric.py:759: RuntimeWarning: invalid value encountered in double_scalars
  ef_a = ss_a / (ss_a + ss_s + ss_as + ss_bs + ss_abs)
c:\Users\User\anaconda3\envs\venv_tremor_music\lib\site-packages\pingouin\parametric.py:760: RuntimeWarning: invalid value encountered in double_scalars
  ef_b = ss_b / (ss_b + ss_s + ss_as + ss_bs + ss_abs)
c:\Users\User\anaconda3\envs\venv_tremor_music\lib\site-pac

In [ ]:
dat_sit_relx["id"].nunique()

39

In [ ]:
dat_sit_flex = sumed_axis[sumed_axis["task"] == "Sitting flexed"]
aov_sit_flex = pg.rm_anova(dv='Norm amp', within=['visit'],
                  subject='id', data=dat_sit_flex)

aov_sit_flex

,Source,ddof1,ddof2,F,p-unc,ng2,eps
0,visit,1,38,3.346308,0.075211,0.042524,1.0


In [ ]:
dat_stand_flex = sumed_axis[sumed_axis["task"] == "Standing flexed"]
aov_sand_flex = pg.rm_anova(dv='Norm amp', within=['visit'],
                  subject='id', data=dat_stand_flex)

aov_sand_flex

,Source,ddof1,ddof2,F,p-unc,ng2,eps
0,visit,1,38,0.793559,0.378632,0.009062,1.0
